In [462]:
import math
import numpy as np
import random

# vypocet funkcni hodnoty f v x
def fx(p, x):
    px = []
    i = 0
    for t in p:
        px.append((t*(x**(len(p)-i-1))))
        i = i + 1
    return sum(px)

# derivace polynomu
def derivation(p):
    derivated=[]
    for i in range(len(p)-1):
        derivated.append((len(p)-i-1)*p[i])
    return derivated

# newtonova metoda pro nalezeni korenu polynomu
def newtons_method(p, x, err, max_iter): 
    xm1 = 0
    i = 0
    while (abs(x - xm1) > err):
        xm1 = x
        xi = x - (fx(p, x)/fx(derivation(p), x))
        x = xi
        i = i + 1
        if i >= max_iter:
            return
    if isinstance(x, complex) and x.imag != 0:
        cx = []
        cx.append(x)  # komplexni koren
        cx.append(-x) # komplexne sdruzeny koren
        return cx
    elif not isinstance(x, complex):
        return x

# metoda bisekce pro nalezeni korene polynomu
def bisection_method(p, a, b, err, max_iter):
    error = abs(b-a)
    i = 0
    
    while error > err:
        c = (b + a) / 2

        if i >= max_iter:
            return
        
        if fx(p, a) * fx(p, b) >= 0:
            #print(f"Na intervalu <{a}, {b}> neleží žádný kořen.")
            #return
            # pokud ma interval ve svych krajnich bodech stejnou funkcni hodnotu, tak jej zacnu zmensovat
            # vim, ze na danem intervalu ten koren lezi (pokud neni komplexni - jinak max_iter)
            if a > b:
                a = a - 1e-3
            else:
                a = a + 1e-3 
            
        elif fx(p, c) * fx(p, a) < 0:
            b = c
            error = abs(b-a)
            
        elif fx(p, c) * fx(p, b) < 0:
            a = c
            error = abs(b-a)
            
        elif fx(p, c) == 0:
            return c  # odhalil jsem koren v c
            
        else:
            print("Chyba bisekce!")
            return 
        i = i + 1
    return (a + b) / 2.0  # vracím průměrnou hodnotu
        
# deleni dvou polynomu
def divide_polynoms(p1, p2):
    pn = []
    q, r = np.polynomial.polynomial.polydiv(p1, p2)
    
    for element in q:
        pn.append(element)
    
    return pn
    #if r[0] == 0 and len(r) == 1:
    #    return q
    #else:
    #    print('Není možné dělit polynomy beze zbytku.')
    #    return [0]

# nalezeni korenu kvadratickeho polynomu
def equation_roots(pq): 
    a = pq[0]
    b = pq[1]
    c = pq[2]
    roots = []
    
    diskriminant = b**2 - 4 * a * c 
    sqrt_val = math.sqrt(abs(diskriminant)) 
    
    # dva realne rozdilne koreny
    if diskriminant > 0: 
        roots.append((-b + sqrt_val)/(2 * a)) 
        roots.append((-b - sqrt_val)/(2 * a)) 
    
    # jeden realny koren
    elif diskriminant == 0: 
        roots.append(-b / (2 * a)) 
    
    # komplexni koreny
    else:
        #roots.append(- b / (2 * a), " + i", sqrt_val) 
        #roots.append(- b / (2 * a), " - i", sqrt_val) 
        roots.append(complex(- b / (2 * a), sqrt_val))
        roots.append(complex(- b / (2 * a), -sqrt_val))
    return roots

# https://is.muni.cz/th/v4vr0/bakalarskaprace.pdf - kapitola 2, věta 2.1
# urceni hranic korenu polynomu
def root_bounds(p):
    A = [abs(a) for a in p]
    A = max(A[1:])

    B = [abs(b) for b in p]
    B = max(B[:len(B)-1])

    an = p[len(p)-1]
    a0 = p[0]

    l_bound = (1 + B / abs(an))**(-1)
    h_bound = (1 + (A / abs(a0)))

    return [l_bound, h_bound]

# generovani x0 pro Newtonovu metodu
def random_x0_in_interval(bounds):
    return random.uniform(bounds[0], bounds[1])

In [463]:
# funkce pro zadani polynomu uzivatelem
def enter_polynom():
    while True:
        try:
            level = int(input("Zadejte stupeň polynomu: "))
            assert 0 < level
        except ValueError:
            print("Zadejte číslo.")
        except AssertionError:
            print("Zadejte číslo větší než 0.")
        else:
            break
    i = 0
    p = []
    while i < level + 1:
        while True:
            try:
                koef = int(input(f"Zadejte koeficient x**{level-i}: "))
                p.append(koef)
                i = i + 1
            except ValueError:
                print("Zadejte číslo.")
            else:
                break
    return p

In [474]:
def main():
    roots = []
    polynoms = []
    err = 1e-20  # presnost
    #p = [1, 0, -5, 0, 4]
    #p = [1, 0, -5, 0, 4, 10]
    p = enter_polynom()
    random_choice = 1
    max_iter = 1000  # maximalni iterace pro Newtonovu metodu - pokud nenalezne s presnosti err, 
                        # tak zkousim znovu s novym x0
    max_random_choice = 100  # maximalni povoleny pocet nahodne zvolenych x0 pro Newtonovu metodu

    # Newton dokud mam polynom vyssiho nez druheho radu
    while len(p) > 3:
        bounds = root_bounds(p)  # odhad intervalu, na jakem se nachazi koreny
        polynoms.append(p)  # pole polynomu, ktere resim
        
        while max_random_choice > random_choice:
            
            x0 = random_x0_in_interval(bounds)
            result = newtons_method(p, x0, err, max_iter)
            
            if result is None:  # provedu znovu nahodny vyber x0 a zvysim random_choice
                random_choice = random_choice + 1
            else:
                break
                
        if random_choice == max_random_choice:  # pouziju metodu bisekce   
            print("Newton: kořen nenalezen.")
            result = bisection_method(p, bounds[0], bounds[1], err, max_iter)
        
        if result is None:  # prerusim pokud ani bisekce nic nenalezne
            print("Bisekce: kořen nenalezen.")
            break
        
        roots.append(result)
        
        # polynom kterym budu delit a ktery obsahuje koren (x - koren)
        dividing_p = (1, - roots[len(roots) - 1])
        pn = divide_polynoms(p, dividing_p)
        
        p = pn

    # na kvadraticky polynom pouziji vzorec
    if len(p) == 3:
        polynoms.append(p)
        for root in equation_roots(pn):
            roots.append(root)
    
    # hledam komplexne sdruzene koreny pomoci Newtonovy metody
    # nepodarilo se mi vyresit nasledne deleni polynomu
    # polydiv mi po deleni polynomem s komplexnim korenem vraci polynom stejneho radu a nasledne naleznu stejne koreny, 
    # ktere jiz mam
    x0 = complex(0, x0)
    result = newtons_method(p, x0, err, max_iter)
    
    if result is not None:
        for root in result:
            roots.append(root)
        
    print(f"Kořeny polynomu jsou: {roots}")
    print(f"Řešené polynomy: {polynoms}")
    
main()

Zadejte stupeň polynomu: 4
Zadejte koeficient x**4: 1
Zadejte koeficient x**3: 0
Zadejte koeficient x**2: -5
Zadejte koeficient x**1: 0
Zadejte koeficient x**0: 4
Kořeny polynomu jsou: [2.0, 1.0, -1.0, -2.0]
Řešené polynomy: [[1, 0, -5, 0, 4], [1.0, 2.0, -1.0, -2.0], [1.0, 3.0, 2.0]]


In [477]:
# ukázka funkčnosti s komplexními kořeny
def main():
    roots = []
    polynoms = []
    err = 1e-20  # presnost
    p = [1, 0, -5, 0, 4, 10]
    random_choice = 1
    max_iter = 1000  # maximalni iterace pro Newtonovu metodu - pokud nenalezne s presnosti err, 
                        # tak zkousim znovu s novym x0
    max_random_choice = 100  # maximalni povoleny pocet nahodne zvolenych x0 pro Newtonovu metodu

    # Newton dokud mam polynom vyssiho nez druheho radu
    while len(p) > 3:
        bounds = root_bounds(p)  # odhad intervalu, na jakem se nachazi koreny
        polynoms.append(p)  # pole polynomu, ktere resim
        
        while max_random_choice > random_choice:
            
            x0 = random_x0_in_interval(bounds)
            result = newtons_method(p, x0, err, max_iter)
            
            if result is None:  # provedu znovu nahodny vyber x0 a zvysim random_choice
                random_choice = random_choice + 1
            else:
                break
                
        if random_choice == max_random_choice:  # pouziju metodu bisekce   
            print("Newton: kořen nenalezen.")
            result = bisection_method(p, bounds[0], bounds[1], err, max_iter)
        
        if result is None:  # prerusim pokud ani bisekce nic nenalezne
            print("Bisekce: kořen nenalezen.")
            break
        
        roots.append(result)
        
        # polynom kterym budu delit a ktery obsahuje koren (x - koren)
        dividing_p = (1, - roots[len(roots) - 1])
        pn = divide_polynoms(p, dividing_p)
        
        p = pn

    # na kvadraticky polynom pouziji vzorec
    if len(p) == 3:
        polynoms.append(p)
        for root in equation_roots(pn):
            roots.append(root)
    
    # hledam komplexne sdruzene koreny pomoci Newtonovy metody
    # nepodarilo se mi vyresit nasledne deleni polynomu
    # polydiv mi po deleni polynomem s komplexnim korenem vraci polynom stejneho radu a nasledne naleznu stejne koreny, 
    # ktere jiz mam
    x0 = complex(0, x0)
    result = newtons_method(p, x0, err, max_iter)
    
    if result is not None:
        for root in result:
            roots.append(root)
        
    print(f"Kořeny polynomu jsou: {roots}")
    print(f"Řešené polynomy: {polynoms}")
    
main()

Newton: kořen nenalezen.
Bisekce: kořen nenalezen.
Kořeny polynomu jsou: [-2.255088411755679, (-0.6502603181222842+0.9370699874491747j), (0.6502603181222842-0.9370699874491747j)]
Řešené polynomy: [[1, 0, -5, 0, 4, 10], [0.9999999999999998, -2.2550884117556786, 0.08542374483475033, -0.19263809706561952, 4.434415940355344]]
